# Labelling experiment results

## Data preperation

In [1]:
import os
import pandas as pd
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np
data_folder = 'data/article_ratings/'
pd.set_option('display.max_rows', 5)

df_total = pd.DataFrame()
for file in os.listdir(data_folder):
    df = pd.read_csv(data_folder + file)
    df["name"] = file.split("_")[0]
    print(f"Number of reviewed articles by {file.split('_')[0]}:    {df.shape[0]}")
    df_total = df_total.append(df)
    


df_art = pd.read_csv("data/articles_topics.csv", index_col = 0)



df_total.rename(columns={"sentiment": "manual_sentiment"}, inplace=True)


df_art = df_art.merge(df_total[["paragraphs", "manual_sentiment", "outcome"]], on="paragraphs", how="left")

df_art["manual_sentiment"] = df_art.manual_sentiment.map({1: "POSITIVE", -1: "NEGATIVE", 0: "NEUTRAL", np.nan: np.nan})

Number of reviewed articles by Ahmed:    28
Number of reviewed articles by Ale:    28
Number of reviewed articles by Artemis:    28
Number of reviewed articles by Bart:    28
Number of reviewed articles by David:    28
Number of reviewed articles by Sophie:    28


C:\Users\barta\AppData\Local\Temp\ipykernel_22440\4071330514.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total = df_total.append(df)
C:\Users\barta\AppData\Local\Temp\ipykernel_22440\4071330514.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total = df_total.append(df)
C:\Users\barta\AppData\Local\Temp\ipykernel_22440\4071330514.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total = df_total.append(df)
C:\Users\barta\AppData\Local\Temp\ipykernel_22440\4071330514.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total = df_total.append(df)
C:\Users\barta\AppData\Local\Temp\ipykernel_22440\4071330514.py:

# Results

### Manual sentiment vs. manual outcome

The p-value shows if there is a association between the two variables. a value lower than 0.05 shows that we reject the $H_0$ hypothesis

In [2]:

df = df_art[df_art["manual_sentiment"].notna()]

# Create a contingency table
contingency_table = pd.crosstab(df['manual_sentiment'], df['outcome'])

# Perform the Chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-squared statistic:", chi2)
print("P-value:", p)
if p < 0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")


Chi-squared statistic: 93.0478643146144
P-value: 2.963639365962942e-19
Reject null hypothesis


### Manual sentiment vs. sentiment from RoBERTa model

In [3]:

df = df_art[df_art["manual_sentiment"].notna()]

# Create a contingency table
contingency_table = pd.crosstab(df['manual_sentiment'], df['sentiment_roberta'])

# Perform the Chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-squared statistic:", chi2)
print("P-value:", p)
if p < 0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

Chi-squared statistic: 29.65399328562594
P-value: 5.755932404002735e-06
Reject null hypothesis


### Sentiment RoBERTa model vs. Manual outcome

In [4]:

df = df_art[df_art["outcome"].notna()]

# Create a contingency table
contingency_table = pd.crosstab(df['sentiment_roberta'], df['outcome'])

# Perform the Chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-squared statistic:", chi2)
print("P-value:", p)
if p < 0.05:
    print("Reject null hypothesis")
else:
    print("Fail to reject null hypothesis")

Chi-squared statistic: 38.0144192000661
P-value: 1.1129105932184073e-07
Reject null hypothesis
